In [ ]:
!pip install riotwatcher
!pip install xlsxwriter
!pip install -q sklearn

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
from riotwatcher import LolWatcher, ApiError
import xlsxwriter
import json
import xlsxwriter as xw
from tqdm import tqdm
from google.colab import files 
import io

In [ ]:
uploaded = files.upload()

Saving trainingnew.xlsx to trainingnew (1).xlsx


In [ ]:
NUMERIC_COLUMNS = ["win", "baw", "bal", "bbw", "bbl", "bcw", "bcl", "bdw", "bdl", "bew", "bel", 
                  "paw", "pal", "pbw", "pbl", "pcw", "pcl", "pdw", "pdl", "pew", "pel"]

RESULTS = ['Team 1 wins', 'Team 2 wins']

train = pd.read_excel('trainingnew (1).xlsx', names=NUMERIC_COLUMNS, header=0)
test = pd.read_excel('testingnew.xlsx', names=NUMERIC_COLUMNS, header=0)
train_y = train.pop('win')
test_y = test.pop('win')

def input_fn(features, labels, training=True, batch_size=20):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

In [ ]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='baw', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bal', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bbw', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bbl', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bcw', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bcl', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bdw', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bdl', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bew', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='bel', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='paw', shape=(1,), de

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns, 
    hidden_units = [30, 10],
    n_classes = 3)

classifier.train(
    input_fn = lambda: input_fn(train, train_y, training=True),
    steps=5000)
clear_output()


In [ ]:
eval_result = classifier.evaluate(input_fn = lambda:input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-13T05:02:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmptzp8n4s6/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.25473s
INFO:tensorflow:Finished evaluation at 2020-09-13-05:02:11
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.48333332, average_loss = 1.457324, global_step = 5000, loss = 1.4573241
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmptzp8n4s6/model.ckpt-5000

Test set accuracy: 0.483

